In [1]:
%matplotlib inline
import csv
import matplotlib.pyplot as plt
import pandas
import scipy

ImportError: No module named csvs

In [ ]:
import hotroutes as H

# Hot Route Analysis and Visualization

## Exploring the data

In [ ]:
d = pandas.read_csv('./rides.csv')
d.head()

In [ ]:
d.describe()

### Plotting with map overlay

In [ ]:
d2 = d[['start_lat', 'start_lng']]
d2.shape

In [ ]:
def plot_pickups():
    plt.figure(figsize=(8, 12))
    d2t = d2.tail(50000)
    plt.scatter(x=d2t.start_lng, y=d2t.start_lat, marker='o', alpha=0.05);
    plt.xlim((-74.025, -73.92))
    plt.ylim((40.65, 40.85))

plot_pickups()

Looks like NYC to me... lat and lng check out

#### A couple of taxi rides...

In [ ]:
d.iloc[0, [2, 3, 5, 6]]

In [ ]:
def plot_rides(rides):
    plt.figure(figsize=(8, 12))
    d2t = d2.tail(50000)
    plt.scatter(x=d2t.start_lng, y=d2t.start_lat, marker='o', alpha=0.05);
    plt.xlim((-74.025, -73.92))
    plt.ylim((40.65, 40.85))
    for [x0, y0, x1, y1] in rides:
        plt.plot([x0, x1], [y0, y1], color='r', linewidth=2)

# lines for rides, let's choose 25
rides = []
for i in range(25):
    ride = d.iloc[i, [2, 3, 5, 6]]
    rides.append([
        ride.start_lng, ride.start_lat, ride.end_lng, ride.end_lat
    ])

plot_rides(rides)

## Load / Generate my solution for 5 best hotroutes

### The bad way

If the file './hotroutes.old.csv' exists, load it. If not, too bad, we're not going to generate it

In [ ]:
def load_hotroutes():
    with open('./hotroutes.old.csv', 'r') as f:
        return list(csv.DictReader(f))

try:
    hr = load_hotroutes()
except IOError:
    hr = []

#### Plot these hotroutes on the same maps

In [ ]:
rides = [
    [row['start_lng'], row['start_lat'], row['end_lng'], row['end_lat']]
    for row in hr
]

plot_rides(rides)

Well that was pretty awful... What happened?

### The Good Way

We have a huge list of pickup and drop off times. Previously, I was randomly generating routes from the square these values were in; why not just choose these values exactly? Bound to get me pretty close to the desired answer.

If the file './hotroutes.csv' exists, load it. If not, generate it by a call to
```python
H.main(nRoutes=10000, nRides=10000)
```

In [ ]:
def load_hotroutes():
    with open('./hotroutes.csv', 'r') as f:
        return list(csv.DictReader(f))

try:
    hr = load_hotroutes()
except IOError:
    H.main(nRoutes=10000, nRides=10000)
    hr = load_hotroutes()

In [ ]:
rides = [
    [row['start_lng'], row['start_lat'], row['end_lng'], row['end_lat']]
    for row in hr
]

plot_rides(rides)

In [ ]:
for row in hr:
    print "({}, {})".format(row['start_lat'], row['start_lng'])
    print "({}, {})".format(row['end_lat'], row['end_lng'])
    print ""

## Final Thoughts

Basically, I think I went about this all the wrong way. There were at least a few things I forgot to take into account:

+ the time it takes to make a trip affects the number of rides (really, should max the rate of rides)
+ a ride should be weighted by the sum of itself and its backward piece
+ by not having a cutoff in the weighting function, I think I probably arbitrarily just chose the center of population as my center point for everything.

Anyway... it was fun! But I don't think I 'got it'.